## Convert PDF to string

In [2]:
import pymupdf

In [3]:
def convert_pdf_to_text(filepath, keyword='holdings'):
    pdf_document = pymupdf.open(filepath)
    text = ''

    for page in pdf_document:
        page_text = page.get_text()
        if keyword.lower() in page_text.lower():
            text += page_text

    pdf_document.close()

    return text

## Extract holdings using AI

##### System prompt & user text

In [38]:
system_prompt = """
You are going to be given a string of text from brokerage statements. Please interpret it to identify the individual positions in the portfolio. This is an example input and output. If you cannot identify a specific attribute, fill it with 'NA'.

Input:
'7 of 28\nINVESTMENT REPORT \nJuly 1 – July 31, 2015\u2009\u2002\u2002\nHoldings (continued)\u2002\nDescription\nQuantity\nPrice \nPer Unit\nEnding \nMarket Value\nTotal \nCost Basis\nUnrealized \nGain/Loss\nEst. Annual \nIncome (EAI)\nEst. Yield \n(EY)\n25.00\n525.31\n$13,132.75 \n$9,350.12 \nc\n$3,782.63 \n$304.68 \n2.32%\nCommon Stocks \nAPPLE INC (AAPL) \nAMERCO COM (UHAL)\n30.00\n203.15A\n 6,094.50 \n 4,149.75 \nc\n 1,944.75 \n—\nTotal Common Stock (24% of account holdings)\n $5,517.25 \n$-1,011.12 \n$6,528.37 \n$304.68 \nAccount 111-111111 \nJohn W. Doe - Individual TOD\n*** SAMPLE STATEMENT ***\nFor informational purposes only\n$304.68 \n'

Output:
{
  "account": {
    "number": "111-111111",
    "name": "John W. Doe - Individual TOD"
  },
  "holdings": [
    {
      "description": "APPLE INC",
      "symbol": "AAPL",
      "quantity": 25.00,
      "total_cost_basis": 9350.12
    },
    {
      "description": "AMERCO COM",
      "symbol": "UHAL",
      "quantity": 30.00,
      "total_cost_basis": 4149.75
    }
  ]
}
"""

In [40]:
test_filepath = r'C:\Users\marco\Desktop\panda\statements\fidelity-sample-statement.pdf'
pdf_document = pymupdf.open(test_filepath)
user_text = pdf_document[20].get_text()

##### OpenAI

In [14]:
from openai import OpenAI

client = OpenAI(organization='org-eVEyczFQOXsiXSb1K0yKkpuz')

In [ ]:
# Make a request to the OpenAI API
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_text}
    ]
)

# Extract and return the JSON response from the model
response['choices'][0]['message']['content']

##### Google

In [ ]:
import google.generativeai as genai
from dotenv import load_dotenv
import os
import json

In [39]:
load_dotenv()
api_key = os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=api_key)

# see https://ai.google.dev/api/python/google/generativeai/GenerativeModel
generation_config = {
  'response_mime_type': 'application/json',
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  system_instruction=system_prompt
)

def extract_holdings(text):
    response = model.generate_content(text)
    return json.loads(response.text)

## Convert json to excel

In [56]:
json_object = {"account": {"number": "111-111111", "name": "John W. Doe - Individual TOD"}, "holdings": [{"description": "DOUBLELINE TOTAL RETURN BOND FD CL I", "symbol": "DBLTX", "quantity": "1,015.328", "total_cost_basis": "7,536.91"}, {"description": "FIDELITY INSTL MMKT PORT CL I", "symbol": "FMPXX", "quantity": "5,025.810", "total_cost_basis": "N/A"}, {"description": "FDIC INSURED DEPOSIT AT FIFTH THIRD BANK IRA NOT COVERED BY SIPC", "symbol": "QPIKQ", "quantity": "3,500.500", "total_cost_basis": "N/A"}, {"description": "SPDR SER TR BARCLAYS CAP HIGH YIELD BD ETF", "symbol": "JNK", "quantity": "250.00", "total_cost_basis": "10,000.00"}, {"description": "VANGUARD INTL EQUITY INDEX FDS MSCI EUROPE ETF", "symbol": "VJK", "quantity": "100.00", "total_cost_basis": "5,000.00"}, {"description": "APPLE INC", "symbol": "AAPL", "quantity": "25.00", "total_cost_basis": "9,350.12"}, {"description": "AMERCO COM", "symbol": "UHAL", "quantity": "30.00", "total_cost_basis": "4,149.75"}, {"description": "ENSTAR GROUP LIMITED COM STK USD 1.00", "symbol": "ESGR", "quantity": "-100.00", "total_cost_basis": "-14,510.99"}, {"description": "BANK AMER CORP DEP SHS REPSTG 1/1200TH PFD SER 5", "symbol": "BMLPRL", "quantity": "150.00", "total_cost_basis": "2,470.16"}, {"description": "CITIGROUP INC DEP SH REP STG 1/100TH INT NON CUMULATIVE PFD SER F 8.50%", "symbol": "CPRM", "quantity": "300.00", "total_cost_basis": "6,895.40"}, {"description": "FEDERAL NATL MTG ASSN", "symbol": "3136FPEX1", "quantity": "10,000.00", "total_cost_basis": "10,250.00"}, {"description": "CALPINE CORP ESCROW 4", "symbol": "131ESC9D9", "quantity": "10,000.000", "total_cost_basis": "10,000.00"}, {"description": "NEW YORK N Y CITY INDL DEV AGY REV", "symbol": "64971PEN1", "quantity": "10,000.000", "total_cost_basis": "10,000.00"}, {"description": "TXU SR NT-P", "symbol": "873168AL2", "quantity": "5,000.00", "total_cost_basis": "10,000.00"}, {"description": "SABRATEK CORP NT CV", "symbol": "78571UAA6", "quantity": "5,000.00", "total_cost_basis": "10,000.00"}, {"description": "UNITED STATES TREAS BILLS", "symbol": "912796BL2", "quantity": "10,000.00", "total_cost_basis": "10,000.00"}, {"description": "ADI NET LEASE INC & GROWTH LP XIX UNIT LTP PARTNERSHIP INT BASED ON MGMTS UNCONFIRMED EST OF NET ASSETS", "symbol": "NA", "quantity": "5.000", "total_cost_basis": "unknown"}, {"description": "VENTAS INC", "symbol": "VTR", "quantity": "500.000", "total_cost_basis": "unknown"}, {"description": "FIDELITY CASH RESERVES", "symbol": "FDRXX", "quantity": "2,070.880", "total_cost_basis": "N/A"}, {"description": "APPLE INC", "symbol": "AAPL", "quantity": "30.00", "total_cost_basis": "8,247.50t"}, {"description": "EXXON MOBILE", "symbol": "XOM", "quantity": "50.00", "total_cost_basis": "4,149.75t"}, {"description": "FACEBOOK", "symbol": "FB", "quantity": "60.00", "total_cost_basis": "1,848.95t"}, {"description": "GENERAL MOTORS CO USD0.01", "symbol": "GM", "quantity": "209.00", "total_cost_basis": "unknown"}, {"description": "SLM CORP CPI LKD NT FLTG RATE 0.00%", "symbol": "OSM", "quantity": "590.00", "total_cost_basis": "9,090.70"}, {"description": "STANDARD PACIFIC CORP", "symbol": "SPF", "quantity": "5,100.00", "total_cost_basis": "61,354.44"}, {"description": "WESTMORELAND COAL CO DEP SHS EACH REPSTG 1/4 OF SHS SER A CV ESCH", "symbol": "WLBPZ", "quantity": "300.00", "total_cost_basis": "5,088.40t"}, {"description": "SABRATEK CORP NT CV", "symbol": "78571UAA6", "quantity": "10000.00", "total_cost_basis": "10,000.00t"}, {"description": "DELPHI CORP JR SB NT", "symbol": "247126AF2", "quantity": "15000.00", "total_cost_basis": "15,025.00"}, {"description": "ESCROW GEN MOTORS CORP BOND", "symbol": "370ESCAT2", "quantity": "10000.00", "total_cost_basis": "9,900.00"}, {"description": "GENERAL MTRS CO WTS EXP 07/10/2016", "symbol": "GMWSA", "quantity": "190.000", "total_cost_basis": "unknown"}, {"description": "GENERAL MTRS CO WTS EXP 07/10/2019", "symbol": "GMWSB", "quantity": "190.000", "total_cost_basis": "unknown"}, {"description": "SOUTHWEST ROYALTIES INSTITUTIONAL INCOME FUND LP IX-B", "symbol": "NA", "quantity": "6.000", "total_cost_basis": "unknown"}, {"description": "NH PORTFOLIO 2015", "symbol": "NA", "quantity": "1,200.291", "total_cost_basis": "NA"}, {"description": "NH MODERATE GROWTH PORT", "symbol": "NA", "quantity": "463.301", "total_cost_basis": "NA"}]}

In [60]:
import pandas as pd 

holdings = json_object['holdings']
df = pd.DataFrame(holdings)

In [62]:
json_object

{'account': {'number': '111-111111', 'name': 'John W. Doe - Individual TOD'},
 'holdings': [{'description': 'DOUBLELINE TOTAL RETURN BOND FD CL I',
   'symbol': 'DBLTX',
   'quantity': '1,015.328',
   'total_cost_basis': '7,536.91'},
  {'description': 'FIDELITY INSTL MMKT PORT CL I',
   'symbol': 'FMPXX',
   'quantity': '5,025.810',
   'total_cost_basis': 'N/A'},
  {'description': 'FDIC INSURED DEPOSIT AT FIFTH THIRD BANK IRA NOT COVERED BY SIPC',
   'symbol': 'QPIKQ',
   'quantity': '3,500.500',
   'total_cost_basis': 'N/A'},
  {'description': 'SPDR SER TR BARCLAYS CAP HIGH YIELD BD ETF',
   'symbol': 'JNK',
   'quantity': '250.00',
   'total_cost_basis': '10,000.00'},
  {'description': 'VANGUARD INTL EQUITY INDEX FDS MSCI EUROPE ETF',
   'symbol': 'VJK',
   'quantity': '100.00',
   'total_cost_basis': '5,000.00'},
  {'description': 'APPLE INC',
   'symbol': 'AAPL',
   'quantity': '25.00',
   'total_cost_basis': '9,350.12'},
  {'description': 'AMERCO COM',
   'symbol': 'UHAL',
   'qua

In [61]:
df

,description,symbol,quantity,total_cost_basis
0,DOUBLELINE TOTAL RETURN BOND FD CL I,DBLTX,"1,015.328","7,536.91"
1,FIDELITY INSTL MMKT PORT CL I,FMPXX,"5,025.810",N/A
2,FDIC INSURED DEPOSIT AT FIFTH THIRD BANK IRA N...,QPIKQ,"3,500.500",N/A
3,SPDR SER TR BARCLAYS CAP HIGH YIELD BD ETF,JNK,250.00,"10,000.00"
4,VANGUARD INTL EQUITY INDEX FDS MSCI EUROPE ETF,VJK,100.00,"5,000.00"
5,APPLE INC,AAPL,25.00,"9,350.12"
6,AMERCO COM,UHAL,30.00,"4,149.75"
7,ENSTAR GROUP LIMITED COM STK USD 1.00,ESGR,-100.00,"-14,510.99"
8,BANK AMER CORP DEP SHS REPSTG 1/1200TH PFD SER 5,BMLPRL,150.00,"2,470.16"
9,CITIGROUP INC DEP SH REP STG 1/100TH INT NON C...,CPRM,300.00,"6,895.40"
